In [1]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta

# Data Loader

In [2]:
class DataLoader:
    req_columns = ['CH_TIMESTAMP', 'CH_SYMBOL', 'CH_TRADE_HIGH_PRICE', 'CH_TRADE_LOW_PRICE', 'CH_OPENING_PRICE', 'CH_CLOSING_PRICE', 'CH_LAST_TRADED_PRICE', 'CH_PREVIOUS_CLS_PRICE', 'CH_TOT_TRADED_QTY', 'CH_52WEEK_HIGH_PRICE', 'CH_52WEEK_LOW_PRICE']
    new_column_names = ['date', 'symbol', 'high', 'low', 'open', 'close', 'ltp', 'prev_close', 'volume', 'high_52w', 'low_52w']

    @staticmethod
    def load_data(script_name, start_date, end_date, series="EQ"):
        df = equity_history(script_name, series, start_date, end_date)[DataLoader.req_columns]
        df.columns = DataLoader.new_column_names
        return df

In [3]:
script_name = 'INFY'
series = 'EQ'
end_date = datetime.now().date()
start_date = end_date - timedelta(days = 365)

In [57]:
df = DataLoader.load_data(
    script_name, 
    start_date.strftime("%d-%m-%Y"), 
    end_date.strftime("%d-%m-%Y"), 
    series).sort_values('date').drop_duplicates()

In [58]:
df['candle_body_length'] = (df['open'] - df['close']).abs()
df['candle_length'] = df['high'] - df['low']
df['candle_body_ratio'] = df['candle_body_length'] / df['candle_length']
df['candle_color'] = np.where(df.close > df.open, 'green', 'red')
df['pct_change'] = (df['close'] - df['prev_close']) * 100 / df['prev_close']

# Multiple candlestick pattern functions

In [87]:
def identify_engulfing(df):
    for i in ['open', 'high', 'close', 'low', 'candle_color', 'candle_body_ratio', 'candle_body_length']:
        df[f'prev_{i}'] = df[i].shift(1)

    df['engulfing'] = np.where(
        np.logical_and.reduce([
            df['prev_candle_color'] != df['candle_color'],
            df['prev_candle_body_ratio'] >= 0.15,
            df['prev_open'].between(df[['open', 'close']].min(axis=1), df[['open', 'close']].max(axis=1)),
            df['prev_close'].between(df[['open', 'close']].min(axis=1), df[['open', 'close']].max(axis=1)),
            (df['prev_candle_body_length'] / df['candle_body_length']) < 0.95
        ]),
        np.where(df["candle_color"] == 'green', "Bullish Engulfing", "Bearish Engulfing"),
        None
    )
    
    return df

def identify_haramis(df):
    df['harami'] = np.where(
        np.logical_and.reduce([
            df['prev_candle_color'] != df['candle_color'],
            df['prev_candle_body_ratio'] >= 0.5,
            (df['candle_body_length'] / df['prev_candle_body_length']) < 0.5,
            df['open'].between(df[['prev_open', 'prev_close']].min(axis=1), df[['prev_open', 'prev_close']].max(axis=1)),
            df['close'].between(df[['prev_open', 'prev_close']].min(axis=1), df[['prev_open', 'prev_close']].max(axis=1)),
        ]),
        np.where(df['candle_color'] == 'green', "Bullish Harami", "Bearish Harami"),
        None
    )
    return df

In [88]:
for fn in [identify_engulfing, identify_haramis]:
    df = fn(df)

In [89]:
df[df['harami'].notnull()]

,date,symbol,high,low,open,close,ltp,prev_close,volume,high_52w,...,candle_color,pct_change,prev_open,prev_high,prev_low,prev_candle_color,prev_candle_body_ratio,prev_candle_body_length,engulfing,harami
249,2021-10-07,INFY,1704.20,1676.65,1687.00,1691.00,1690.35,1673.55,5476108,1788.0,...,green,1.042694,1702.10,1709.00,1670.00,red,0.732051,28.55,None,Bullish Harami
179,2021-12-07,INFY,1724.90,1694.00,1701.10,1711.65,1711.45,1695.30,4202122,1848.0,...,green,0.964431,1730.00,1733.95,1691.50,red,0.817432,34.70,None,Bullish Harami
160,2022-01-28,INFY,1727.55,1679.50,1681.00,1686.20,1685.80,1678.60,10998502,1953.9,...,green,0.452758,1702.00,1709.70,1665.00,red,0.523490,23.40,None,Bullish Harami
117,2022-02-16,INFY,1755.45,1726.85,1738.25,1733.30,1731.50,1738.25,5350642,1953.9,...,red,-0.284769,1699.00,1746.75,1690.45,green,0.697158,39.25,None,Bearish Harami
126,2022-03-02,INFY,1714.40,1691.00,1710.00,1702.80,1702.55,1715.60,6771489,1953.9,...,red,-0.746095,1682.20,1728.05,1665.00,green,0.529738,33.40,None,Bearish Harami
105,2022-04-20,INFY,1596.30,1563.00,1575.00,1587.70,1592.20,1562.00,10678747,1953.9,...,green,1.645327,1636.65,1636.65,1550.00,red,0.861512,74.65,None,Bullish Harami
72,2022-05-20,INFY,1466.75,1446.70,1453.00,1455.15,1459.00,1427.15,10488908,1953.9,...,green,1.961952,1481.00,1484.70,1417.65,red,0.803132,53.85,None,Bullish Harami
76,2022-05-26,INFY,1430.00,1407.05,1418.00,1423.95,1427.95,1410.45,13040065,1953.9,...,green,0.957141,1435.00,1442.50,1399.25,red,0.567630,24.55,None,Bullish Harami
79,2022-05-31,INFY,1518.00,1488.15,1510.70,1503.60,1510.00,1526.80,16033857,1953.9,...,red,-1.519518,1487.00,1530.00,1485.00,green,0.884444,39.80,None,Bearish Harami
47,2022-07-06,INFY,1500.00,1470.00,1482.20,1491.65,1491.90,1475.95,5209162,1953.9,...,green,1.063722,1498.80,1509.95,1472.00,red,0.602108,22.85,None,Bullish Harami
